In [ ]:
from PIL import Image
from os import getcwd

from common.utils import * 

%matplotlib inline

input_dir = getcwd() + '/img/input_samples/'
outp_dir = getcwd() + '/img/output_fft_noise_images/'

# FFT Spectrum leakage

In [ ]:
size = 100

noise = np.random.normal(0, 1, size)
noise = noise - np.mean(noise)

zero_padded = np.copy(noise)
zero_padded[-1] = zero_padded[0] 
zero_padded = zero_padded - np.mean(zero_padded)

noise_img = find_ft_1d(noise)
zero_padded_img = find_ft_1d(zero_padded)

x = freq_numbers_1d(size)
f = freq_pink_filter_1d(x)
f = normalize(f)

noise_mod = adjust_img_1d(noise)
noise_img_mod = adjust_img_1d(noise_img)
zero_padded_img_mod = adjust_img_1d(zero_padded_img)
x_mod = adjust_freq_1d(x)
f_mod = adjust_freq_1d(f)


fig = plt.figure(figsize=(10, 4))
plt.plot(noise)
plt.plot(zero_padded)
plt.grid(True)

In [ ]:
fig = plt.figure(figsize=(10, 4))

diff = (-np.abs(noise_img_mod) + np.abs(zero_padded_img_mod)) / np.max(np.abs(noise_img_mod)) * 100 

plt.plot(x_mod, np.abs(noise_img_mod))
plt.plot(x_mod, np.abs(zero_padded_img_mod))
plt.grid(True)

In [ ]:
fig = plt.figure(figsize=(12, 4))
ax1 = fig.add_subplot(1, 2, 1)
im1 = ax1.plot(noise)
im2 = ax1.plot(zero_padded, '--')
plt.grid()
plt.xlabel('а')

ax2 = fig.add_subplot(1, 2, 2)
im2 = ax2.plot(x_mod, np.abs(noise_img_mod))
plt.plot(x_mod, np.abs(zero_padded_img_mod), '--')
plt.xlabel('б')

# plt.savefig(outp_dir + 'spectrum_leakage.png')
plt.grid()
plt.show()

# FFT Synthesis demo

In [ ]:
size = 100

noise = np.random.normal(0, 1, size)
noise[-1] = noise[0] 
noise = noise - np.mean(noise)

zero_padded = np.copy(noise)
zero_padded[-1] = zero_padded[0] 
zero_padded = zero_padded - np.mean(zero_padded)

noise_img = find_ft_1d(noise)
zero_padded_img = find_ft_1d(zero_padded)

x = freq_numbers_1d(size)
f = freq_pink_filter_1d(x, factor=1)
f = normalize(f)

noise_mod = adjust_img_1d(noise)
noise_img_mod = adjust_img_1d(noise_img)
zero_padded_img_mod = adjust_img_1d(zero_padded_img)
x_mod = adjust_freq_1d(x)
f_mod = adjust_freq_1d(f)

plt.plot(x_mod, noise_mod)
plt.plot(x_mod, f_mod, 'g--')
plt.plot(x_mod, (f_mod * noise_mod))
plt.grid()

In [ ]:
init_power = sum(abs(noise_img) ** 2)
pink_power = sum(abs(noise_img * f) ** 2)
power_factor = (init_power / pink_power) ** 0.5
pink_noise = power_factor * find_ift_1d(noise_img * f).real

plt.plot(noise)
plt.plot(pink_noise, '--')
plt.grid()

In [ ]:
fig = plt.figure(figsize=(12, 4))
ax1 = fig.add_subplot(1, 2, 1)
im1 = ax1.plot(noise)
im2 = ax1.plot(pink_noise, '--')
plt.grid()
plt.xlabel('а')

ax2 = fig.add_subplot(1, 2, 2)
im1 = ax2.plot(x_mod, noise_img_mod.real)
im2 = ax2.plot(x_mod, 10 * f_mod, 'g--')
im3 = ax2.plot(x_mod, (f_mod * noise_img_mod.real), '--')
plt.grid()
plt.xlabel('б')

# plt.savefig(outp_dir + 'Fourier_synthesis.png')
plt.show()

In [ ]:
plt.loglog(x, np.abs(noise_img))
plt.loglog(x, np.abs(noise_img * f))
plt.loglog(x, f)
plt.gca().set_ybound(10 ** -3, 100)

plt.grid()

# 2D FFT Synthesis demo

In [ ]:
size = (128, 128)

noise = np.random.normal(0, 1, size)
noise = noise - np.mean(noise)
noise_img = find_ft_2d(noise)

x = freq_numbers_1d(size[0])
y = freq_numbers_1d(size[1])
f_1 = freq_pink_filter_2d(x, y, factor=1)
f_1 = normalize(f_1)

# need to add naive stretching along axes
# f_2 = freq_filter_2d(x, y, x_aspect=1.5, y_aspect=0.5, factor=1)
f_2 = freq_pink_filter_2d(x, y, factor=1.5)
f_2 = normalize(f_2)

X, Y = np.meshgrid(x, y)

pic_1 = find_ift_2d(noise_img * f_1).real
pic_1 = normalize(pic_1)

pic_2 = find_ift_2d(noise_img * f_2).real
pic_2 = normalize(pic_2)

In [ ]:
fig = plt.figure(figsize=(10, 4))
ax1 = fig.add_subplot(1, 2, 1)
im1 = ax1.imshow(pic_1, cmap='gray')
plt.xlabel('а')

ax2 = fig.add_subplot(1, 2, 2)
im2 = ax2.imshow(pic_2, cmap='gray')
plt.xlabel('б')

cax = fig.add_axes([0.9, 0.11, 0.02, 0.77]) 
cbar = plt.colorbar(im1, cax=cax)

plt.subplots_adjust(wspace=0)
# plt.savefig(outp_dir + 'fourier_synthesis_2d_case.png')
plt.show()

In [ ]:
fig, ax = plt.subplots(subplot_kw={"projection": "3d"})
ax.plot_surface(X, Y, np.log(f_1))
ax.plot_surface(X, Y, np.log(f_2))

In [ ]:
fig = plt.figure(figsize=(10, 4))
ax1 = fig.add_subplot(1, 2, 1)
im1 = ax1.imshow(np.log(f_1), cmap='gray')
plt.xlabel('а')

ax2 = fig.add_subplot(1, 2, 2)
im2 = ax2.imshow(np.log(f_2), cmap='gray')
plt.xlabel('б')

cax = fig.add_axes([0.9, 0.11, 0.02, 0.77]) 
cbar = plt.colorbar(im1, cax=cax)

plt.subplots_adjust(wspace=0)
# plt.savefig(outp_dir + 'value_noise_2d_case.png')
plt.show()

# Rubik's cube example

In [ ]:
cube = Image.open(input_dir +'cube_1.png').convert('L')
cube_fr = find_ft_2d(cube)

y_size, x_size = cube_fr.shape
xx = np.linspace(-x_size / 2, x_size / 2, x_size)
yy = np.linspace(-y_size / 2, y_size / 2, y_size)

kernel = normalize(freq_sharp_round_filter_2d(xx, yy, 50, False))
eps = 10 ** -10

f, (ax1, ax2) = plt.subplots(1, 2, sharey=True, figsize=(10, 10))
ax1.imshow(cube, cmap='gray', vmin=0, vmax=255)

# Wrong scaling, but it gives a really interesting visual representation
ax2.imshow(find_ift_2d(cube_fr * kernel).real, cmap='gray', vmin=0, vmax=np.max(np.log(np.absolute(cube_fr + eps))))

In [ ]:
cube = Image.open(input_dir + 'woman.jpeg').convert('L')
cube_fr = find_ft_2d(cube)

y_size, x_size = cube_fr.shape
xx = np.linspace(-x_size / 2, x_size / 2, x_size)
yy = np.linspace(-y_size / 2, y_size / 2, y_size)

fps = 30
seconds_num = 5
x_mesh, y_mesh = np.meshgrid(xx, yy)

radius = 300
cube_freq = [cube]
kernel_arr = []

for i in range(fps * seconds_num):
    kernel = freq_sharp_round_filter_2d(xx, yy, i / fps / seconds_num * 150, False)
    kernel_arr.append(kernel)
    cube_freq.append(find_ift_2d(cube_fr * kernel).real)

In [ ]:
%matplotlib notebook

fig = plt.figure(figsize=(8,8))
im = plt.imshow(cube_freq[0], interpolation='none', aspect='equal', cmap='gray')


def animate_func(i): 
    im.set_array(cube_freq[i])
    return [im]


anim = animation.FuncAnimation(
    fig, 
    animate_func, 
    frames = seconds_num * fps,
    interval = 1000 / fps, # in ms
)


# anim.save('women.gif', writer='pillow')

In [ ]:
%matplotlib inline

cube = Image.open(input_dir + 'cube_1.png').convert('L')
cube_fr = find_ft(cube)

eps = 10 ** -10

f, (ax1, ax2) = plt.subplots(1, 2, sharey=True, figsize=(10, 10))
ax1.imshow(cube, cmap='gray', vmin=0, vmax=255)
ax2.imshow(np.log(np.absolute(cube_fr + eps)), cmap='gray')

# Phase correlation example

In [ ]:
f1 = Image.open(input_dir + 'horse_1.png').convert('L')
f2 = Image.open(input_dir + 'horse_translated_1.png').convert('L')

f1_freq = find_ft_2d(f1)
f2_freq = find_ft_2d(f2)

f, (ax1, ax2) = plt.subplots(1, 2, sharey=True, figsize=(15, 15))
ax1.imshow(f1, cmap='gray', vmin=0, vmax=255)
ax2.imshow(f2, cmap='gray', vmin=0, vmax=255)

In [ ]:
f, (ax1, ax2) = plt.subplots(1, 2, sharey=True, figsize=(15, 15))
ax1.imshow(np.log(np.absolute(f1_freq)), cmap='gray')
ax2.imshow(np.log(np.absolute(f2_freq)), cmap='gray')

In [ ]:
f, (ax1, ax2) = plt.subplots(1, 2, sharey=True, figsize=(15, 15))
ax1.imshow(np.angle(f1_freq, deg=True), cmap='gray')
ax2.imshow(np.angle(f2_freq, deg=True), cmap='gray')

In [ ]:
ncps = f1_freq * np.conj(f2_freq) / np.abs(f1_freq * f2_freq)
shift = find_ift_2d(ncps)

f, (ax1, ax2) = plt.subplots(1, 2, sharey=True, figsize=(15, 15))
ax1.imshow(np.absolute(ncps), cmap='gray')
ax2.imshow(np.absolute(shift), cmap='gray')

In [ ]:
ind = np.unravel_index(np.argmax(shift, axis=None), shift.shape)
[i - j for i, j in zip(shift.shape, ind)]

# Shifting Pink Noise 

In [ ]:
x_size = 256
y_size = 256

xx = np.linspace(-x_size / 2, x_size / 2, x_size)
yy = np.linspace(-y_size / 2, y_size / 2, y_size)

# Generating cloud image
whitenoise = np.random.normal(0, 1, (y_size, x_size))
ft_arr = find_ft_2d(whitenoise)
kernel = freq_pink_filter_2d(xx, yy, factor=1.4)

pink_ft_arr = ft_arr * kernel
pink_noise = normalize_img(find_ift_2d(pink_ft_arr).real)

# Shifting initial image
dx = 25
dy = 20

x_mesh, y_mesh = np.meshgrid(xx, yy)
shift = np.exp(-1j * 2 * np.pi  * (x_mesh * dx / y_size + y_mesh * dy / x_size));

# Apply the phase shift along both axes
f_freq_shifted = pink_ft_arr * shift
shifted_pink_noise = normalize_img(find_ift_2d(f_freq_shifted).real)

show_images(pink_noise, shifted_pink_noise)

In [ ]:
%matplotlib notebook

# Generating cloud image
whitenoise = np.random.normal(0, 1, (y_size, x_size))
ft_arr = find_ft_2d(whitenoise)
kernel = freq_pink_filter_2d(xx, yy, factor=2)

pink_ft_arr = ft_arr * kernel
pink_noise = find_ift_2d(pink_ft_arr).real

# Shift values
dx = 200
dy = 0

# Shift values per one frame
fps = 30
seconds_num = 5
dxx = dx / fps / seconds_num
dyy = dy / fps / seconds_num

x_mesh, y_mesh = np.meshgrid(xx, yy)
shift = np.exp(-1j * 2 * np.pi  * (x_mesh * dxx / y_size + y_mesh * dyy / x_size));

snapshots_freq = [pink_ft_arr]
snapshots_spatial = [pink_noise]
for _ in range(fps * seconds_num):
    # print(np.sum(np.absolute(snapshots_freq[-1]) ** 2))
    snapshots_freq.append(snapshots_freq[-1] * shift)
    snapshots_spatial.append(find_ift_2d(snapshots_freq[-1]).real)

    
fig = plt.figure(figsize=(8,8))
frame = snapshots_spatial[0]
im = plt.imshow(frame, interpolation='none', aspect='auto', cmap='gray')


def animate_func(i):
    im.set_array(snapshots_spatial[i])
    plt.title(f"X shift {i * dxx:.2f} \n Y shift {i * dyy:.2f}")
    return [im]


anim = animation.FuncAnimation(
    fig, 
    animate_func, 
    frames = seconds_num * fps,
    interval = 1000 / fps, # in ms
)

# Smooth transition

In [ ]:
%matplotlib inline

x_size = 256
y_size = 256

xx = np.linspace(-x_size / 2, x_size / 2, x_size)
yy = np.linspace(-y_size / 2, y_size / 2, y_size)  
depth = int(x_size * 0.3)

img_1 = gen_cloud(x_size, y_size)
img_2 = gen_cloud(x_size, y_size)

img_1_cut = img_1[:, -depth:x_size]
img_2_cut = img_2[:, 0:depth]
img_2_tr = img_2[:,depth:x_size]

x_kernel = spatial_smooth_filter(x_size, y_size, depth)
img1_cut_pr = img_1_cut * x_kernel + img_2_cut * (1 - x_kernel)
img_1[:,-depth:x_size] = img1_cut_pr
img_concat = np.concatenate((img_1, img_2_tr), axis=1)

show_images(img_1, img_2)

In [ ]:
plt.plot(range(depth), x_kernel[0])

In [ ]:
show_images(img_concat)

In [ ]:
img = gen_cloud(x_size, y_size)
img_new, add_img = make_img_transition_x(img, depth)
img_concat = np.concatenate((img_new, add_img), axis=1)

show_images(img, img_new, add_img, img_concat)

In [ ]:
img = gen_cloud(x_size, y_size)
img_new, add_img = make_img_transition_x(img, depth, is_dx_pos=False)
img_concat = np.concatenate((add_img, img_new), axis=1)

show_images(img, img_new, add_img, img_concat)

In [ ]:
img = gen_cloud(x_size, y_size)
img_new, add_img = make_img_transition_y(img, depth)
img_concat = np.concatenate((img_new, add_img), axis=0)

show_images(img, img_new, add_img, img_concat)

In [ ]:
img = gen_cloud(x_size, y_size)
img_new, add_img = make_img_transition_y(img, depth, is_dy_pos=False)
img_concat = np.concatenate((add_img, img_new), axis=0)

show_images(img, img_new, add_img, img_concat)

In [ ]:
img = gen_cloud(x_size, y_size)

img_new, img_x = make_img_transition_x(img, depth)
img_row_1 = np.concatenate((img_new, img_x), axis=1)

img_new, img_y = make_img_transition_y(img_row_1, depth)
img_row_2 = np.concatenate((img_new, img_y), axis=0)

show_images(img_row_1, img_row_2)

In [ ]:
img = gen_cloud(x_size, y_size)

img_new, img_x = make_img_transition_x(img, depth, is_dx_pos=False)
img_row_1 = np.concatenate((img_x, img_new), axis=1)

img_new, img_y = make_img_transition_y(img_row_1, depth)
img_row_2 = np.concatenate((img_new, img_y), axis=0)

show_images(img_row_1, img_row_2)

In [ ]:
img = gen_cloud(x_size, y_size)

img_new, img_x = make_img_transition_x(img, depth)
img_row_1 = np.concatenate((img_new, img_x), axis=1)

img_new, img_y = make_img_transition_y(img_row_1, depth, is_dy_pos=False)
img_row_2 = np.concatenate((img_y, img_new), axis=0)

show_images(img_row_1, img_row_2)

In [ ]:
img = gen_cloud(x_size, y_size)

img_new, img_x = make_img_transition_x(img, depth, is_dx_pos=False)
img_row_1 = np.concatenate((img_x, img_new), axis=1)

img_new, img_y = make_img_transition_y(img_row_1, depth, is_dy_pos=False)
img_row_2 = np.concatenate((img_y, img_new), axis=0)

show_images(img_row_1, img_row_2)

# Example of XY smooth transition and shift

In [ ]:
%matplotlib inline

x_size = 256
y_size = 256
depth = int(x_size * 0.3)

xx = np.linspace(-x_size / 2, x_size / 2, x_size)
yy = np.linspace(-y_size / 2, y_size / 2, y_size)  

img = gen_cloud(x_size, y_size)

new_img = make_img_transition_xy(img, depth)
shifted_img = shift_img_xy(new_img, img.shape, 75, 50)

show_images(new_img)

In [ ]:
show_images(shifted_img)

In [ ]:
%matplotlib notebook

# Shift values
dx = 200
dy = 200

# Shift values per one frame
fps = 30
seconds_num = 5
dxx = dx / fps / seconds_num
dyy = dy / fps / seconds_num

dx_arr = [0]
dy_arr = [0]
for i in range(fps * seconds_num):
    dx_arr.append(dx_arr[-1] + dxx)
    dy_arr.append(dy_arr[-1] + dyy)
    

for i in range(fps * seconds_num):
    dx_arr[i] = round(dx_arr[i])
    dy_arr[i] = round(dy_arr[i])
    

x_mesh, y_mesh = np.meshgrid(xx, yy)
snapshots = []
for i in range(fps * seconds_num):
    snapshots.append(shift_img_xy(new_img, (256, 256), dx_arr[i], dy_arr[i]))

    
fig = plt.figure(figsize=(5,5))
frame = snapshots[0]
im = plt.imshow(frame, interpolation='none', aspect='auto', cmap='gray')


def animate_func(i):
    im.set_array(snapshots[i])
    plt.title(f"X shift {i * dxx:.2f} \n Y shift {i * dyy:.2f}")
    return [im]


anim = animation.FuncAnimation(
    fig, 
    animate_func, 
    frames = seconds_num * fps,
    interval = 1000 / fps, # in ms
)